In [1]:
import requests
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
import matplotlib.ticker as ticker

In [2]:
# We are using cleaned data in order to make the process quicker
df = pd.read_csv(r"C:\Users\Gor088\Downloads\311_Service_Requests_2023_Summer - Cleaned.csv", low_memory=False)

In [ ]:
df.head()

In [ ]:
df.columns

In [5]:
df = df[['Created Date', 'Agency', 'Complaint Type', 'Descriptor',
         'Incident Zip', 'Closed Date', 'Borough',
         'Status', 'Latitude', 'Longitude']]

In [ ]:
df

In [ ]:
print(df['Created Date'].isna().sum())
print(df['Agency'].isna().sum())
print(df['Complaint Type'].isna().sum())
print(df['Descriptor'].isna().sum())
print(df['Incident Zip'].isna().sum())
print(df['Closed Date'].isna().sum())
print(df['Borough'].isna().sum())
print(df['Status'].isna().sum())
print(df['Latitude'].isna().sum())
print(df['Longitude'].isna().sum())

In [8]:
df = df.dropna(subset=['Closed Date'])
df = df.dropna(subset=['Incident Zip'])
df = df.dropna(subset=['Descriptor'])
df = df.dropna(subset=['Longitude'])

In [ ]:
df

In [ ]:
df['Borough'].unique()

In [12]:
df = df[df['Borough'] != 'Unspecified']

In [13]:
#df['Status'].unique()

In [14]:
#df['Status'].value_counts()

In [15]:
#df = df[df['Status'] == 'Closed']

In [16]:
df = df.reset_index(drop=True)

In [ ]:
df['Created Date'] = pd.to_datetime(df['Created Date'])
df['Closed Date'] = pd.to_datetime(df['Closed Date'])
df['Resolution Time'] = (df['Closed Date'] - df['Created Date']).dt.total_seconds() / 60
df['Incident Zip'] = df['Incident Zip'].astype(int)

In [ ]:
df

In [ ]:
plt.figure(figsize=(10, 6))
df['Borough'].value_counts().plot(kind='bar', title='Complaints by Borough')
plt.show()

In [ ]:
average_resolution_time_per_agency = df.groupby('Borough')['Resolution Time'].mean().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
average_resolution_time_per_agency.plot(kind='bar', title='Average Resolution Time Per Borough')

plt.xlabel('Borough')
plt.ylabel('Average Resolution Time (minutes)')

plt.show()

In [ ]:
# top 10 most common complaint types
top_complaints = df['Complaint Type'].value_counts().head(10)

plt.figure(figsize=(10, 6))
top_complaints.plot(kind='bar', title='Most Common Complaints')

plt.show()

In [ ]:
average_resolution_time_per_agency = df.groupby('Agency')['Resolution Time'].mean().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
average_resolution_time_per_agency.plot(kind='bar', title='Average Resolution Time Per Agency')

plt.xlabel('Agency')
plt.ylabel('Average Resolution Time (minutes)')

plt.show()

In [ ]:
# top 10 most common complaint types
top_complaints = df['Complaint Type'].value_counts().head(10)

plt.figure(figsize=(10, 6))
top_complaints.plot(kind='bar', title='Most Common Complaints')

plt.show()

In [ ]:
top_agencie = df['Agency'].value_counts().head(10)

plt.figure(figsize=(10, 6))
top_agencie.plot(kind='bar', title = 'Most Complaints by Agency')

plt.show()

In [ ]:
average_resolution_time_per_agency = df.groupby('Agency')['Resolution Time'].mean().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
average_resolution_time_per_agency.plot(kind='bar', title='Average Resolution Time by Agency')

plt.xlabel('Agency')
plt.ylabel('Average Resolution Time (minutes)')

plt.show()

In [ ]:
m = folium.Map(location=[40.73, -73.92], zoom_start=8)

# Prepare data for HeatMap (each entry: [latitude, longitude, intensity])
heat_data = [[row['Latitude'], row['Longitude'], 1] for index, row in df.iterrows()]  # Each point is treated with intensity 1

# Add heat map to the folium map
HeatMap(heat_data).add_to(m)

# If you're in a Jupyter notebook, just display the map
m

In [ ]:
top_5_agencies = df['Agency'].value_counts().head(5).index

df_top5 = df[df['Agency'].isin(top_5_agencies)].copy()

df_top5.loc[:, 'Hour'] = pd.to_datetime(df_top5['Created Date']).dt.hour

calls_by_hour_agency = df_top5.groupby(['Hour', 'Agency']).size().reset_index(name='total_calls')

pivot_calls = calls_by_hour_agency.pivot(index='Hour', columns='Agency', values='total_calls')

plt.figure(figsize=(12, 6))
pivot_calls.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.title('Total Calls by Hour and Top 5 Agencies')
plt.xlabel('Hour of the Day')
plt.ylabel('Total Calls')
plt.legend(title='Agency', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
pivot_calls = calls_by_hour_agency.pivot(index='Hour', columns='Agency', values='total_calls').fillna(0)

plt.figure(figsize=(12, 6))
pivot_calls.plot(kind='area', stacked=True, alpha=0.8)

plt.title('Total Calls by Hour and Top 5 Agencies (Stacked Area Plot)')
plt.xlabel('Hour of the Day')
plt.ylabel('Total Calls')
plt.tight_layout()
plt.show()

In [26]:
# df.to_csv('311_Service_Requests_2023_Summer.csv', index=False)